# Test de Migration - Nouvelle Structure Modulaire

Ce notebook teste la nouvelle structure modulaire du projet STA211.

## 1. Test de la configuration

In [ ]:
# Import de la configuration
import sys
sys.path.append('..')

from modules.config import cfg

print("✅ Configuration chargée")
print(f"📁 Racine du projet: {cfg.root}")
print(f"🏷️ Nom du projet: {cfg.project.project_name}")
print(f"👤 Auteur: {cfg.project.author}")
print(f"📊 Random state: {cfg.project.random_state}")

print("\n📂 Chemins configurés:")
for attr_name in ['data', 'raw', 'processed', 'outputs', 'figures', 'models', 'imputers', 'transformers']:
    path = getattr(cfg.paths, attr_name)
    exists = "✅" if path.exists() else "❌"
    print(f"  {exists} {attr_name}: {path}")

## 2. Test des utilitaires de stockage

In [ ]:
from modules.utils import save_artifact, load_artifact, artifact_exists
import numpy as np

print("✅ Utilitaires de stockage importés")

# Test de sauvegarde/rechargement
test_data = {'test': 'data', 'array': np.array([1, 2, 3])}
test_filename = "test_artifact.pkl"

# Sauvegarder
saved_path = save_artifact(test_data, test_filename, cfg.paths.models)
print(f"💾 Données sauvegardées: {saved_path}")

# Vérifier existence
exists = artifact_exists(test_filename, cfg.paths.models)
print(f"🔍 Artefact existe: {exists}")

# Recharger
loaded_data = load_artifact(test_filename, cfg.paths.models)
print(f"📂 Données rechargées: {loaded_data}")

# Vérifier intégrité
integrity_check = (
    loaded_data['test'] == test_data['test'] and 
    np.array_equal(loaded_data['array'], test_data['array'])
)
print(f"✅ Intégrité des données: {integrity_check}")

## 3. Test du module de prétraitement

In [ ]:
from modules.notebook1_preprocessing import (
    load_and_clean_data,
    perform_knn_imputation,
    apply_optimal_transformations,
    detect_and_cap_outliers
)

print("✅ Module de prétraitement importé")
print("📦 Fonctions disponibles:")
print("  - load_and_clean_data")
print("  - perform_knn_imputation")
print("  - apply_optimal_transformations")
print("  - detect_and_cap_outliers")

# Test avec des données factices
import pandas as pd
np.random.seed(42)

# Créer des données de test
n_samples = 100
test_df = pd.DataFrame({
    'feature1': np.random.normal(0, 1, n_samples),
    'feature2': np.random.normal(5, 2, n_samples),
    'target': np.random.binomial(1, 0.3, n_samples)
})

# Introduire quelques valeurs manquantes
missing_idx = np.random.choice(n_samples, 10, replace=False)
test_df.loc[missing_idx, 'feature1'] = np.nan

print(f"\n📊 Données de test créées: {test_df.shape}")
print(f"🕳️ Valeurs manquantes: {test_df.isnull().sum().sum()}")

# Test KNN imputation
df_imputed, imputer = perform_knn_imputation(
    test_df, ['feature1'], n_neighbors=5, save_imputer=False
)

remaining_missing = df_imputed.isnull().sum().sum()
print(f"✅ Imputation KNN: {remaining_missing} valeurs manquantes restantes")

## 4. Test du module de modélisation

In [ ]:
from modules.notebook2_modeling import (
    get_default_param_grids,
    create_model_estimators,
    optimize_classification_threshold
)

print("✅ Module de modélisation importé")

# Test des grilles de paramètres
param_grids = get_default_param_grids()
print(f"\n📋 Grilles de paramètres disponibles: {list(param_grids.keys())}")

# Test des estimateurs
estimators = create_model_estimators()
print(f"🤖 Estimateurs disponibles: {list(estimators.keys())}")

# Test d'optimisation de seuil avec des données factices
y_true = np.random.binomial(1, 0.3, 1000)
y_proba = np.random.beta(2, 5, 1000)  # Probabilités factices

threshold_results = optimize_classification_threshold(
    y_true, y_proba, metric='f1', verbose=False
)

print(f"\n🎯 Seuil optimal: {threshold_results['best_threshold']:.3f}")
print(f"📈 Score F1 optimal: {threshold_results['best_score']:.3f}")

## 5. Test du module d'ensembles

In [ ]:
from modules.modeling import (
    create_voting_ensemble,
    create_stacking_ensemble,
    get_ensemble_feature_importance
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

print("✅ Module d'ensembles importé")

# Créer des modèles de base pour test
base_models = {
    'rf': RandomForestClassifier(n_estimators=10, random_state=42),
    'svm': SVC(probability=True, random_state=42),
    'lr': LogisticRegression(random_state=42)
}

# Test Voting Ensemble
voting_ensemble = create_voting_ensemble(base_models, voting='soft')
print(f"🗳️ Voting Ensemble créé: {type(voting_ensemble).__name__}")

# Test Stacking Ensemble
stacking_ensemble = create_stacking_ensemble(base_models)
print(f"🥞 Stacking Ensemble créé: {type(stacking_ensemble).__name__}")

print("\n📦 Fonctions d'ensemble disponibles:")
print("  - create_voting_ensemble")
print("  - create_bagging_ensemble")
print("  - create_stacking_ensemble")
print("  - optimize_ensemble")
print("  - train_all_ensembles")

## 6. Test du module d'évaluation

In [ ]:
from modules.evaluation import (
    calculate_basic_metrics,
    calculate_detailed_metrics,
    optimize_threshold,
    generate_evaluation_report
)

print("✅ Module d'évaluation importé")

# Test avec des prédictions factices
y_true = np.random.binomial(1, 0.4, 200)
y_pred = np.random.binomial(1, 0.4, 200)
y_proba = np.random.beta(2, 3, 200)

# Test des métriques de base
basic_metrics = calculate_basic_metrics(y_true, y_pred, y_proba)
print(f"\n📊 Métriques de base calculées: {list(basic_metrics.keys())}")
print(f"   F1-Score: {basic_metrics['f1']:.3f}")
print(f"   AUC-ROC: {basic_metrics.get('auc_roc', 'N/A')}")

# Test des métriques détaillées
detailed_metrics = calculate_detailed_metrics(y_true, y_pred, y_proba)
print(f"\n📈 Métriques détaillées disponibles: {list(detailed_metrics.keys())}")

# Test d'optimisation de seuil
threshold_opt = optimize_threshold(y_true, y_proba, metric='f1')
print(f"\n🎯 Optimisation de seuil:")
print(f"   Seuil optimal: {threshold_opt['best_threshold']:.3f}")
print(f"   Score optimal: {threshold_opt['best_score']:.3f}")

print("\n📋 Fonctions d'évaluation disponibles:")
print("  - calculate_basic_metrics")
print("  - calculate_detailed_metrics") 
print("  - optimize_threshold")
print("  - analyze_threshold_sensitivity")
print("  - plot_evaluation_dashboard")
print("  - generate_evaluation_report")

## 7. Test de génération de rapport

In [ ]:
# Simulation des résultats d'un modèle
model_results = {
    'basic_metrics': basic_metrics,
    'classification_report': detailed_metrics['classification_report'],
    'confusion_matrix': detailed_metrics['confusion_matrix'],
    'optimal_threshold': threshold_opt,
    'training_time': 45.2,
    'best_cv_score': 0.742
}

# Générer un rapport
report = generate_evaluation_report(
    model_results, 
    model_name="TestModel", 
    save_report=False
)

print("📝 Rapport d'évaluation généré:")
print("=" * 50)
print(report[:500] + "..." if len(report) > 500 else report)

## 8. Résumé des tests

In [ ]:
print("🎉 RÉSUMÉ DES TESTS DE MIGRATION")
print("=" * 50)
print("✅ Configuration centralisée")
print("✅ Utilitaires de stockage")
print("✅ Module de prétraitement (Notebook 1)")
print("✅ Module de modélisation (Notebook 2)")
print("✅ Module d'ensembles (Notebook 3)")
print("✅ Module d'évaluation")
print("✅ Génération de rapports")
print("\n🚀 La migration vers la nouvelle structure modulaire est RÉUSSIE !")
print("\n📖 Consultez MIGRATION_GUIDE.md pour les détails d'utilisation.")